In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,30241
2,Huelva,Confirmados PDIA 14 días,1267
3,Huelva,Tasa PDIA 14 días,"246,89673987177738"
4,Huelva,Confirmados PDIA 7 días,612
5,Huelva,Total Confirmados,30420
6,Huelva,Curados,27732
7,Huelva,Fallecidos,377


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  30241.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7916.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 888 personas en los últimos 7 días 

Un positivo PCR cada 396 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,30241.0,612.0,1267.0,513170.0,119.258725,246.896740,50.0
Huelva-Costa,17459.0,272.0,591.0,289548.0,93.939520,204.111235,32.0
Huelva (capital),7916.0,162.0,363.0,143837.0,112.627488,252.369001,23.0
Condado-Campiña,9766.0,292.0,576.0,156231.0,186.902727,368.684832,15.0
Sierra de Huelva-Andévalo Central,2672.0,45.0,97.0,67391.0,66.774495,143.936134,3.0
Punta Umbría,784.0,27.0,66.0,15355.0,175.838489,429.827418,3.0
Palos de la Frontera,619.0,12.0,22.0,11742.0,102.197241,187.361608,3.0
Almonte,1730.0,69.0,146.0,24507.0,281.552210,595.748154,3.0
Trigueros,291.0,12.0,19.0,7862.0,152.632918,241.668787,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,163.0,13.0,16.0,2211.0,587.969245,723.654455,0.0
Palma del Condado (La),1115.0,43.0,70.0,10801.0,398.111286,648.088140,2.0
Bollullos Par del Condado,944.0,44.0,88.0,14387.0,305.831654,611.663307,1.0
Almonte,1730.0,69.0,146.0,24507.0,281.552210,595.748154,3.0
Cabezas Rubias,36.0,4.0,4.0,706.0,566.572238,566.572238,0.0
Moguer,1215.0,53.0,98.0,21867.0,242.374354,448.163900,0.0
Punta Umbría,784.0,27.0,66.0,15355.0,175.838489,429.827418,3.0
Calañas,185.0,8.0,11.0,2768.0,289.017341,397.398844,0.0
Valverde del Camino,530.0,24.0,49.0,12750.0,188.235294,384.313725,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villalba del Alcor,480.0,2.0,9.0,3366.0,59.417706,267.379679,0.0,0.222222
Niebla,211.0,4.0,15.0,4116.0,97.181730,364.431487,0.0,0.266667
Lepe,2107.0,6.0,22.0,27880.0,21.520803,78.909613,1.0,0.272727
Lucena del Puerto,243.0,3.0,10.0,3261.0,91.996320,306.654400,0.0,0.300000
Rociana del Condado,538.0,4.0,13.0,7939.0,50.384179,163.748583,0.0,0.307692
Isla Cristina,2129.0,16.0,50.0,21393.0,74.790819,233.721311,0.0,0.320000
Beas,205.0,1.0,3.0,4341.0,23.036167,69.108500,1.0,0.333333
Nerva,173.0,4.0,11.0,5169.0,77.384407,212.807119,0.0,0.363636
San Juan del Puerto,477.0,9.0,23.0,9411.0,95.632770,244.394857,2.0,0.391304
